<a href="https://colab.research.google.com/github/cmannnn/california_housing/blob/main/california_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Housing Corp. - California Housing

The first task you are asked to perform is to build a model of housing prices in California using the California census data. This data has metrics such as the population, median income, median housing price, and so on for each block group in California. Block groups are the smallest geographical unit for which the US Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people). We will just call them “districts” for short.

**Your task is to predict median house values in Californian districts, given a number of features from these districts.**

#Imports

In [1]:
# imports
import sys
print(f'Using sys version: {sys.version_info}')

import requests

import csv

import os

import tarfile

import urllib

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')
%matplotlib inline
print(f'Using matplotlib version: {matplotlib.__version__}')

import numpy as np
print(f'Using numpy version: {np.__version__}')

import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
print(f'Using pandas version: {pd.__version__}')

import seaborn as sns
print(f'Using seaborn version: {sns.__version__}')

import warnings
warnings.simplefilter('ignore')

Using sys version: sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
Using matplotlib version: 3.2.2
Using numpy version: 1.19.4
Using pandas version: 1.1.5
Using seaborn version: 0.11.0


In [2]:
# mounting google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
# using zipfile and URLLIB to download link 
DOWNLOAD_ROOT = ('https://www.dcc.fc.up.pt/~ltorgo/')
HOUSING_PATH = os.path.join('datasets', 'housing')
HOUSING_URL = DOWNLOAD_ROOT + 'Regression/cal_housing.tgz'

In [12]:
def fetch_housing_data(housing_url = HOUSING_URL, housing_path = HOUSING_PATH):
  if not os.path.isdir(housing_path): 
    os.makedirs(housing_path)
  tgz_path = os.path.join(housing_path, 'housing.tgz')
  urllib.request.urlretrieve(housing_url, tgz_path)
  housing_tgz = tarfile.open(tgz_path)
  housing_tgz.extractall(path=housing_path)
  housing_tgz.close()

In [13]:
fetch_housing_data()

In [18]:
# opening housing data
def load_housing_data(housing_path = HOUSING_PATH):
  csv_path = os.path.join(housing_path, 'housing.csv')
  return pd.read_csv(csv_path)

In [ ]:
housing_df = load_housing_data()
housing_df.head()

In [ ]:
housing = load_housing_data()
housing.head()